In [2]:
import numpy as np
import pandas as pd
import os 
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import matplotlib.pyplot as plt 

from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from torch.utils.data.dataloader import DataLoader 

In [3]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device('cpu')

device



device(type='cuda')

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision import transforms
import os
from PIL import Image
import numpy as np
import time


# 1. BASE CNN MODEL 
class CancerCNN(nn.Module):
    def __init__(self, num_classes=5):
        super(CancerCNN, self).__init__()
        
        # RULE: Each conv layer maintains spatial size with padding=1
        # RULE: MaxPool2d(kernel_size=2, stride=2) halves the spatial dimensions
        
        # Input: 3 channels (RGB)
        
        # Conv Block 1
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)  # 128x128 -> 128x128 (padding keeps size)
        self.bn1 = nn.BatchNorm2d(32)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)  # 128x128 -> 64x64
        
        # Conv Block 2
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)  # 64x64 -> 64x64
        self.bn2 = nn.BatchNorm2d(64)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)  # 64x64 -> 32x32
        
        # Conv Block 3
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)  # 32x32 -> 32x32
        self.bn3 = nn.BatchNorm2d(128)
        self.relu3 = nn.ReLU()
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)  # 32x32 -> 16x16
        
        # Conv Block 4
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, padding=1)  # 16x16 -> 16x16
        self.bn4 = nn.BatchNorm2d(256)
        self.relu4 = nn.ReLU()
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2)  # 16x16 -> 8x8
        
        #256 channels * 8 * 8 = 16384
        self.flatten = nn.Flatten()
        
        # Fully Connected Layers
        self.fc1 = nn.Linear(256 * 8 * 8, 512)  # 16384 -> 512
        self.dropout1 = nn.Dropout(0.5)
        self.relu_fc1 = nn.ReLU()
        
        self.fc2 = nn.Linear(512, 256)
        self.dropout2 = nn.Dropout(0.3)
        self.relu_fc2 = nn.ReLU()
        
        self.fc3 = nn.Linear(256, num_classes)  # Final output
        
    def forward(self, x):
        # Conv Block 1
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        
        # Conv Block 2
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu2(x)
        x = self.pool2(x)
        
        # Conv Block 3
        x = self.conv3(x)
        x = self.bn3(x)
        x = self.relu3(x)
        x = self.pool3(x)
        
        # Conv Block 4
        x = self.conv4(x)
        x = self.bn4(x)
        x = self.relu4(x)
        x = self.pool4(x)
        
        # Flatten
        x = self.flatten(x)
        
        # FC Layers
        x = self.fc1(x)
        x = self.dropout1(x)
        x = self.relu_fc1(x)
        
        x = self.fc2(x)
        x = self.dropout2(x)
        x = self.relu_fc2(x)
        
        x = self.fc3(x)
        
        return x


# 2. DATASET CLASS

class CancerDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.classes = ['colon_aca', 'colon_n', 'lung_aca', 'lung_n', 'lung_scc']
        self.class_to_idx = {cls: idx for idx, cls in enumerate(self.classes)}
        self.samples = []
        
        print(f"✓ Root directory: {root_dir}")
        
        for class_name in self.classes:
            class_dir = os.path.join(root_dir, class_name)
            if os.path.exists(class_dir):
                images = [f for f in os.listdir(class_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.tif', '.tiff'))]
                for img_name in images[:5000]:  # Used to check if loading works on smaller samples
                    img_path = os.path.join(class_dir, img_name)
                    self.samples.append((img_path, self.class_to_idx[class_name]))
                print(f"  {class_name}: {len(images)} images")
        
        print(f"Total samples loaded: {len(self.samples)}")
    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        image = Image.open(img_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
        
        return image, label


# 3. TRAINING FUNCTION 
def train_with_immediate_output():
      
    print("Traisn MODEL")
     
    
    # Setup
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"\n[SETUP] Device: {device}")
    if device.type == 'cuda':
        print(f"[SETUP] GPU: {torch.cuda.get_device_name(0)}")
        print(f"[SETUP] CUDA Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
    
    # Define transforms
    transform = transforms.Compose([
        transforms.Resize((128, 128)),  # Fixed size
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])
    
    # Load dataset
    print("\n[DATA] Loading dataset...")
    data_dir = "lungcolon"
    
    if not os.path.exists(data_dir):
        print(f"ERROR: Dataset not found at '{data_dir}'")

        return
    
    # Create dataset
    dataset = CancerDataset(root_dir=data_dir, transform=transform)
    
    # Split dataset (70% train, 15% val, 15% test)
    train_size = int(0.7 * len(dataset))
    val_size = int(0.15 * len(dataset))
    test_size = len(dataset) - train_size - val_size
    
    train_dataset, val_dataset, test_dataset = random_split(
        dataset, [train_size, val_size, test_size],
        generator=torch.Generator().manual_seed(42)
    )
    
    print(f"\n[DATA] Dataset splits:")
    print(f"  Training: {len(train_dataset)} samples")
    print(f"  Validation: {len(val_dataset)} samples")
    print(f"  Test: {len(test_dataset)} samples")
    print(f"  Classes: {dataset.classes}")
    
    # Create data loaders
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)  # num_workers=0 for immediate output
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=0)
    
    # Create model
    model = CancerCNN(num_classes=len(dataset.classes)).to(device)
    
      
    print("MODEL ARCHITECTURE")
     
    print(model)
    
    # Calculate parameters
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"\n[MODEL] Total parameters: {total_params:,}")
    print(f"[MODEL] Trainable parameters: {trainable_params:,}")
    
    # Training setup
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
      
    print("TRAINING")
     
    
    
    # Training loop
    num_epochs = 20  # Start with 3 epochs for quick results IF CPU it will take hors 
    model.train()
    
    for epoch in range(num_epochs):
        epoch_start_time = time.time()
        running_loss = 0.0
        correct = 0
        total = 0
        
        print(f"\n[EPOCH {epoch+1}/{num_epochs}] Starting...")
    
        
        # Training batches
        for batch_idx, (images, labels) in enumerate(train_loader):
            batch_start_time = time.time()
            
            # Move to device
            images, labels = images.to(device), labels.to(device)
            
            # Forward pass
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            # Backward pass
            loss.backward()
            optimizer.step()
            
            # Calculate accuracy
            _, predicted = torch.max(outputs.data, 1)
            batch_total = labels.size(0)
            batch_correct = (predicted == labels).sum().item()
            
            # Update statistics
            running_loss += loss.item()
            total += batch_total
            correct += batch_correct
            
            # Print immediate progress
            batch_time = time.time() - batch_start_time
            
            if batch_idx == 0:
                print(f"[BATCH 1] Loss: {loss.item():.4f}, Acc: {100*batch_correct/batch_total:.1f}%, Time: {batch_time:.1f}s")
                print(f"[INFO] First batch completed! Training continues...")
            elif (batch_idx + 1) % 10 == 0:  # Print every 10 batches
                avg_loss = running_loss / (batch_idx + 1)
                current_acc = 100 * correct / total
                print(f"[BATCH {batch_idx+1}] Avg Loss: {avg_loss:.4f}, Current Acc: {current_acc:.1f}%")
        
        # Epoch statistics
        epoch_time = time.time() - epoch_start_time
        epoch_loss = running_loss / len(train_loader)
        epoch_acc = 100 * correct / total
        
        print(f"\n[EPOCH {epoch+1} SUMMARY]")
        print(f"  Loss: {epoch_loss:.4f}")
        print(f"  Accuracy: {epoch_acc:.2f}%")
        print(f"  Time: {epoch_time:.1f} seconds")
        
        # Validation
        model.eval()
        val_correct = 0
        val_total = 0
        val_loss = 0.0
        
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()
        
        val_acc = 100 * val_correct / val_total
        avg_val_loss = val_loss / len(val_loader)
        
        print(f"[VALIDATION] Loss: {avg_val_loss:.4f}, Accuracy: {val_acc:.2f}%")
        
        
        model.train()  # Set back to training mode
    
    # Save model
    torch.save({
        'model_state_dict': model.state_dict(),
        'epochs': num_epochs,
        'loss': epoch_loss,
        'accuracy': epoch_acc,
        'classes': dataset.classes
    }, 'cancer_cnn_model.pth')
    
    print(f"\n" + "="*70)
    print("TRAINING COMPLETE!")
     
    print(f"Model saved as 'cancer_cnn_model.pth'")
    print(f"Final Training Accuracy: {epoch_acc:.2f}%")
    print(f"Final Validation Accuracy: {val_acc:.2f}%")
    print(f"Total training time: {time.time() - start_time:.1f} seconds")
    
    return model

# 4.  TEST FUNCTION

def quick_test():
    """Quick test to verify everything works before full training"""
      
    print("QUICK TEST - Verifying CNN connections")
     
    
    # Create a simple model
    model = CancerCNN(num_classes=5)
    
    # Test forward pass
    print("\n[TEST] Testing forward pass with dummy data...")
    dummy_input = torch.randn(2, 3, 128, 128)  # Batch of 2, 3 channels, 128x128
    
    print(f"[TEST] Input shape: {dummy_input.shape}")
    
    # Test each layer
    x = dummy_input
    print(f"\n[TEST] Layer-by-layer shape transformation:")
    print(f"  Input: {x.shape}")
    
    # Conv Block 1
    x = model.conv1(x)
    print(f"  After conv1: {x.shape}")
    x = model.bn1(x)
    x = model.relu1(x)
    x = model.pool1(x)
    print(f"  After pool1: {x.shape}")
    
    # Conv Block 2
    x = model.conv2(x)
    print(f"  After conv2: {x.shape}")
    x = model.bn2(x)
    x = model.relu2(x)
    x = model.pool2(x)
    print(f"  After pool2: {x.shape}")
    
    # Conv Block 3
    x = model.conv3(x)
    print(f"  After conv3: {x.shape}")
    x = model.bn3(x)
    x = model.relu3(x)
    x = model.pool3(x)
    print(f"  After pool3: {x.shape}")
    
    # Conv Block 4
    x = model.conv4(x)
    print(f"  After conv4: {x.shape}")
    x = model.bn4(x)
    x = model.relu4(x)
    x = model.pool4(x)
    print(f"  After pool4: {x.shape}")
    
    # Flatten
    x = model.flatten(x)
    print(f"  After flatten: {x.shape}")
    
    # Full forward pass
    print(f"\n[TEST] Full forward pass...")
    output = model(dummy_input)
    print(f"[TEST] Output shape: {output.shape}")
    print(f"[TEST] ✓ CNN connections are correct!")
    
    # Check kernel sizes
    print(f"\n[TEST] Kernel sizes:")
    print(f"  conv1: {model.conv1.kernel_size}")
    print(f"  conv2: {model.conv2.kernel_size}")
    print(f"  conv3: {model.conv3.kernel_size}")
    print(f"  conv4: {model.conv4.kernel_size}")
    
    return True


# 5. MAIN 
if __name__ == "__main__":
    start_time = time.time()
    
      
    print("CANCER HISTOPATHOLOGY CNN CLASSIFIER")
     
    
    # Step 1: Quick test
    print("\n Running quick CNN connection test...")
    quick_test()
    
    # Step 2: Training
    print("\n Starting actual training...")
    
    
    try:
        model = train_with_immediate_output()
        
          
        
    except Exception as e:
       print (f"ERROR")

CANCER HISTOPATHOLOGY CNN CLASSIFIER

[STEP 1] Running quick CNN connection test...
QUICK TEST - Verifying CNN connections

[TEST] Testing forward pass with dummy data...
[TEST] Input shape: torch.Size([2, 3, 128, 128])

[TEST] Layer-by-layer shape transformation:
  Input: torch.Size([2, 3, 128, 128])
  After conv1: torch.Size([2, 32, 128, 128])
  After pool1: torch.Size([2, 32, 64, 64])
  After conv2: torch.Size([2, 64, 64, 64])
  After pool2: torch.Size([2, 64, 32, 32])
  After conv3: torch.Size([2, 128, 32, 32])
  After pool3: torch.Size([2, 128, 16, 16])
  After conv4: torch.Size([2, 256, 16, 16])
  After pool4: torch.Size([2, 256, 8, 8])
  After flatten: torch.Size([2, 16384])

[TEST] Full forward pass...
[TEST] Output shape: torch.Size([2, 5])
[TEST] ✓ CNN connections are correct!

[TEST] Kernel sizes:
  conv1: (3, 3)
  conv2: (3, 3)
  conv3: (3, 3)
  conv4: (3, 3)

[STEP 2] Starting actual training...
NOTE: You should see output within 30-60 seconds!
Traisn MODEL

[SETUP] Device